[![Script](https://acm.im/bristlecone//img/badge-script.svg)](https://acm.im/bristlecone//examples/predator-prey.fsx)&emsp;
[![Notebook](https://acm.im/bristlecone//img/badge-notebook.svg)](https://acm.im/bristlecone//examples/predator-prey.ipynb)

## Predator-Prey Dynamics: Snowshoe Hare and Lynx

Here we use the classic example of snowshoe hare and lynx predator-prey dynamics,
to demonstrate the basic functions of Bristlecone. The dataset is a 90-year
time-series of snowshoe hare and lynx pelts purchased by the
Hudson's Bay Company of Canada. Data is in 1000s.

To get started, we first load and open the Bristlecone library in
an F# script file (.fsx):



In [2]:
open Bristlecone // Opens Bristlecone core library and estimation engine
open Bristlecone.Language // Open the language for writing Bristlecone models


### Defining the ecological model

In Bristlecone, a single ecological model (representing a single hypothesis)
is defined through the `ModelSystem` type. A `ModelSystem` needs to include three key
components:

* **Model equations.** When working in continuous time, these are a system of Ordinary Differential Equations (ODEs).

* **Parameters to be estimated.** You must specify the starting bounds and constraints for every parameter included in the model equations.

* **Likelihood function**. The (negative log) likelihood function **-logL** represents the probability of observing the data given the parameter set. We use a negative log likelihood function, which is then minimised during optimisation.

In this example, we demonstrate using the **Lotka–Volterra** predator–prey model as the
model system. For the -logL function we use a bivariate normal negative log likelihood function.
This -logL function assumes normally-distributed observation error around each observation
at each time-point, for both the lynx and hare data. The -logL function contains
three parameters that are to be estimated alongside the deterministic model: the variability
in lynx data, the variability in hare data, and their covariance.



In [3]:
let ``predator-prey`` =

    let ``dh/dt`` = Parameter "α" * This - Parameter "β" * This * Environment "lynx"
    let ``dl/dt`` = -Parameter "δ" * This + Parameter "γ" * Environment "hare" * This

    Model.empty
    |> Model.addEquation "hare" ``dh/dt``
    |> Model.addEquation "lynx" ``dl/dt``

    |> Model.estimateParameter "α" noConstraints 0.75 1.25 // Natural growth rate of hares in absence of predation
    |> Model.estimateParameter "β" noConstraints 0.01 0.20 // Death rate per encounter of hares due to predation
    |> Model.estimateParameter "δ" noConstraints 0.75 1.25 // Natural death rate of lynx in the absence of food
    |> Model.estimateParameter "γ" noConstraints 0.01 0.20 // Efficiency of turning predated hares into lynx

    |> Model.useLikelihoodFunction (ModelLibrary.Likelihood.bivariateGaussian "hare" "lynx")
    |> Model.estimateParameter "ρ" noConstraints -0.500 0.500
    |> Model.estimateParameter "σ[x]" notNegative 0.001 0.100
    |> Model.estimateParameter "σ[y]" notNegative 0.001 0.100

    |> Model.compile


### Setting up the **Bristlecone Engine**

A bristlecone engine provides a fixed setup for estimating parameters from data.
Use the same engine for all model fits within a single study.
This engine uses a gradident descent method (Nelder Mead simplex), and a basic
Runge-Kutta 4 integration method provided by MathNet Numerics.



In [4]:
let engine =
    Bristlecone.mkContinuous
    // |> Bristlecone.withCustomOptimisation (Optimisation.Amoeba.swarm 5 20 Optimisation.Amoeba.Solver.Default)
    |> Bristlecone.withCustomOptimisation (
        Optimisation.MonteCarlo.Filzbach.filzbach
            { Optimisation.MonteCarlo.Filzbach.FilzbachSettings<float>.Default with
                BurnLength = Optimisation.EndConditions.afterIteration 10000 }
    )
    |> Bristlecone.withContinuousTime Integration.MathNet.integrate
    |> Bristlecone.withConditioning Conditioning.RepeatFirstDataPoint
    |> Bristlecone.withSeed 1000 // We are setting a seed for this example - see below


**Note. We have set a seed for random number generation for this worked example. This ensures that the results are the same each time this documentation is generated.**

### Does it all work? Testing the engine and model

Before being confident in the ability of our estimation engine to
be able to arrive at the correct solution, we must run a full test
of the model and estimation engine.

Bristlecone includes the `Bristlecone.testModel` function, which
we use here. Given a model system and estimation engine, the function
generates a random parameter set (**θ**) many times; for each **θ**, the
'true' time-series are generated. The test result indicates the effectiveness
of the configuration at estimating **θ** given the auto-generated
time-series. If there is divergence, there is likely an
issue with your model or the Bristlecone configuration.

Bristlecone includes many settings to configure the test
procedure. A simple test configuration is set as `Test.defaultSettings`,
but here we will configure some additional settings:



In [5]:
let testSettings =
    Test.create
    |> Test.addStartValues [ "hare", 50.; "lynx", 75. ]
    |> Test.addNoise (Test.Noise.tryAddNormal "σ[y]" "lynx")
    |> Test.addNoise (Test.Noise.tryAddNormal "σ[x]" "hare")
    |> Test.addGenerationRules
        [ Test.GenerationRules.alwaysLessThan 100000. "lynx"
          Test.GenerationRules.alwaysMoreThan 10. "lynx"
          Test.GenerationRules.alwaysLessThan 100000. "hare"
          Test.GenerationRules.alwaysMoreThan 10. "hare" ]
    |> Test.withTimeSeriesLength 30
    |> Test.endWhen (Optimisation.EndConditions.afterIteration 100)


In our `TestSettings`, we have specified the initial time point (t = 0)
for both modelled time-series. We have also added noise around
each generated time-series, and specified that each time-series
should be 30 years in length.

With these test settings, we can now run the test.



In [6]:
let testResult = ``predator-prey`` |> Bristlecone.tryTestModel engine testSettings


##1## GeneralEvent "Attempting to generate parameter set."##1## GeneralEvent "The data must comply with 4 rules after 50000 tries."##1## GeneralEvent  "Time-series start at 1970/01/01 with resolution Fixed (Years PositiveInt 1)."##1## DebugEvent  "Observations occur on a fixed temporal resolution: Years PositiveInt 1."##1## DebugEvent  "No environmental forcing data was supplied. Solving using time points of observations."##1## GeneralEvent  "Skipping optimisation: only the result of the given parameters will be computed"##1## DebugEvent  "Observations occur on a fixed temporal resolution: Years PositiveInt 1."##1## DebugEvent  "No environmental forcing data was supplied. Solving using time points of observations."##1## DebugEvent  "Observations occur on a fixed temporal resolution: Years PositiveInt 1."##1## DebugEvent  "No environmental forcing data was supplied. Solving using time points of observations."##1## GeneralEvent  "Time-series start at 1970/01/01 with resolution Fixed (Yea

We can plot the test results to check the fit.

<div><div id="b0ac8c02-5183-4806-8e61-945e6561d243"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_b0ac8c02518348068e61945e6561d243 = function() {
    var data = [{"type":"scatter","name":"Modelled","mode":"lines","x":["1970-01-01T00:00:00","1971-01-01T00:00:00","1972-01-01T00:00:00","1973-01-01T00:00:00","1974-01-01T00:00:00","1975-01-01T00:00:00","1976-01-01T00:00:00","1977-01-01T00:00:00","1978-01-01T00:00:00","1979-01-01T00:00:00","1980-01-01T00:00:00","1981-01-01T00:00:00","1982-01-01T00:00:00","1983-01-01T00:00:00","1984-01-01T00:00:00","1985-01-01T00:00:00","1986-01-01T00:00:00","1987-01-01T00:00:00","1988-01-01T00:00:00","1989-01-01T00:00:00","1990-01-01T00:00:00","1991-01-01T00:00:00","1992-01-01T00:00:00","1993-01-01T00:00:00","1994-01-01T00:00:00","1995-01-01T00:00:00","1996-01-01T00:00:00","1997-01-01T00:00:00","1998-01-01T00:00:00","1999-01-01T00:00:00","2000-01-01T00:00:00"],"y":[48.720871681360265,53.72566358685745,58.776657422693205,56.276413056876244,50.354083952419174,49.293477280772365,53.821370632308295,58.19758401767473,55.89350426389379,50.67257247305542,49.804718363459884,53.89681172814508,57.6829126845519,55.559493306336634,50.95795244686737,50.26011797510644,53.95410406786743,57.225171014634384,55.26928517853271,51.21470376186428,50.66494040484609,53.99557070122519,56.817922897608966,55.01796955388492,51.44639141723271,51.02413295269137,54.02353265365327,56.455561122552034,54.80096086539102,51.65590725990684,51.34229631785992],"marker":{},"line":{},"xaxis":"x","yaxis":"y"},{"type":"scatter","name":"Observed","mode":"lines","x":["1970-01-01T00:00:00","1971-01-01T00:00:00","1972-01-01T00:00:00","1973-01-01T00:00:00","1974-01-01T00:00:00","1975-01-01T00:00:00","1976-01-01T00:00:00","1977-01-01T00:00:00","1978-01-01T00:00:00","1979-01-01T00:00:00","1980-01-01T00:00:00","1981-01-01T00:00:00","1982-01-01T00:00:00","1983-01-01T00:00:00","1984-01-01T00:00:00","1985-01-01T00:00:00","1986-01-01T00:00:00","1987-01-01T00:00:00","1988-01-01T00:00:00","1989-01-01T00:00:00","1990-01-01T00:00:00","1991-01-01T00:00:00","1992-01-01T00:00:00","1993-01-01T00:00:00","1994-01-01T00:00:00","1995-01-01T00:00:00","1996-01-01T00:00:00","1997-01-01T00:00:00","1998-01-01T00:00:00","1999-01-01T00:00:00","2000-01-01T00:00:00"],"y":[49.9967576146405,48.261137807277166,53.9688227647262,63.16568939261545,67.14415003541156,60.21602698591815,51.039687909528844,48.43867157062676,53.20414804516046,61.91362117174448,66.80588579261814,61.222041367523936,52.117715758377585,48.68039347038799,52.553276784814166,60.80903403236736,66.28657007329697,62.04711514587841,53.17061501470611,49.020917401812504,52.01617753858883,59.70291609604999,65.6877860346433,62.721235317985105,54.232727392672494,49.49580299002807,51.62234301016524,58.6940402053183,65.02097424939036,63.23997659611803,55.25028036764773],"marker":{},"line":{},"xaxis":"x","yaxis":"y"},{"type":"scatter","name":"Modelled","mode":"lines","x":["1970-01-01T00:00:00","1971-01-01T00:00:00","1972-01-01T00:00:00","1973-01-01T00:00:00","1974-01-01T00:00:00","1975-01-01T00:00:00","1976-01-01T00:00:00","1977-01-01T00:00:00","1978-01-01T00:00:00","1979-01-01T00:00:00","1980-01-01T00:00:00","1981-01-01T00:00:00","1982-01-01T00:00:00","1983-01-01T00:00:00","1984-01-01T00:00:00","1985-01-01T00:00:00","1986-01-01T00:00:00","1987-01-01T00:00:00","1988-01-01T00:00:00","1989-01-01T00:00:00","1990-01-01T00:00:00","1991-01-01T00:00:00","1992-01-01T00:00:00","1993-01-01T00:00:00","1994-01-01T00:00:00","1995-01-01T00:00:00","1996-01-01T00:00:00","1997-01-01T00:00:00","1998-01-01T00:00:00","1999-01-01T00:00:00","2000-01-01T00:00:00"],"y":[66.78953059771278,62.69159591093249,67.52596115741794,75.26171461081776,74.33262349223831,66.99756690816929,63.449563826518194,67.88393421756365,74.64116900515947,73.70126519155308,67.18593506786192,64.1279512380381,68.18928761781412,74.08840770577791,73.14554926005178,67.35865440910912,64.73436199120701,68.44792071113206,73.5949656979125,72.6575914272442,67.51842542337786,65.27575927169181,68.66554924721835,73.15382826144767,72.22999081454363,67.66706499428501,65.75851925996143,68.84751851781279,72.75907253176796,71.85593022648715,67.80580761949754],"marker":{},"line":{},"xaxis":"x2","yaxis":"y2"},{"type":"scatter","name":"Observed","mode":"lines","x":["1970-01-01T00:00:00","1971-01-01T00:00:00","1972-01-01T00:00:00","1973-01-01T00:00:00","1974-01-01T00:00:00","1975-01-01T00:00:00","1976-01-01T00:00:00","1977-01-01T00:00:00","1978-01-01T00:00:00","1979-01-01T00:00:00","1980-01-01T00:00:00","1981-01-01T00:00:00","1982-01-01T00:00:00","1983-01-01T00:00:00","1984-01-01T00:00:00","1985-01-01T00:00:00","1986-01-01T00:00:00","1987-01-01T00:00:00","1988-01-01T00:00:00","1989-01-01T00:00:00","1990-01-01T00:00:00","1991-01-01T00:00:00","1992-01-01T00:00:00","1993-01-01T00:00:00","1994-01-01T00:00:00","1995-01-01T00:00:00","1996-01-01T00:00:00","1997-01-01T00:00:00","1998-01-01T00:00:00","1999-01-01T00:00:00","2000-01-01T00:00:00"],"y":[75.04778037226114,63.903133137658614,56.576186057812116,58.22461955998248,68.69821694221238,78.42488925002255,75.71688413474675,65.20492687118224,57.367386526310156,57.936874985219944,67.1047639786269,77.33910873982435,76.11937297399552,66.44631353643553,58.31659158763647,57.8790979774312,65.93368642783169,76.08676347554216,76.54816235563916,67.84684104053332,59.42093565363402,57.84881439445027,64.75739011435337,74.79634052782694,76.76351552125361,68.86557234347423,60.302089149227875,57.99465307595784,63.67729238992065,73.67746231531873,76.68390768006567],"marker":{},"line":{},"xaxis":"x2","yaxis":"y2"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"title":{"x":0.05},"font":{"color":"rgba(42, 63, 95, 1.0)"},"paper_bgcolor":"rgba(255, 255, 255, 1.0)","plot_bgcolor":"rgba(229, 236, 246, 1.0)","autotypenumbers":"strict","colorscale":{"diverging":[[0.0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1.0,"#276419"]],"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]},"hovermode":"closest","hoverlabel":{"align":"left"},"coloraxis":{"colorbar":{"outlinewidth":0.0,"ticks":""}},"geo":{"showland":true,"landcolor":"rgba(229, 236, 246, 1.0)","showlakes":true,"lakecolor":"rgba(255, 255, 255, 1.0)","subunitcolor":"rgba(255, 255, 255, 1.0)","bgcolor":"rgba(255, 255, 255, 1.0)"},"mapbox":{"style":"light"},"polar":{"bgcolor":"rgba(229, 236, 246, 1.0)","radialaxis":{"linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","ticks":""},"angularaxis":{"linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","ticks":""}},"scene":{"xaxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","gridwidth":2.0,"zerolinecolor":"rgba(255, 255, 255, 1.0)","backgroundcolor":"rgba(229, 236, 246, 1.0)","showbackground":true},"yaxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","gridwidth":2.0,"zerolinecolor":"rgba(255, 255, 255, 1.0)","backgroundcolor":"rgba(229, 236, 246, 1.0)","showbackground":true},"zaxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","gridwidth":2.0,"zerolinecolor":"rgba(255, 255, 255, 1.0)","backgroundcolor":"rgba(229, 236, 246, 1.0)","showbackground":true}},"ternary":{"aaxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)"},"baxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)"},"caxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)"},"bgcolor":"rgba(229, 236, 246, 1.0)"},"xaxis":{"title":{"standoff":15},"ticks":"","automargin":"height+width+left+right+top+bottom","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","zerolinecolor":"rgba(255, 255, 255, 1.0)","zerolinewidth":2.0},"yaxis":{"title":{"standoff":15},"ticks":"","automargin":"height+width+left+right+top+bottom","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","zerolinecolor":"rgba(255, 255, 255, 1.0)","zerolinewidth":2.0},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"shapedefaults":{"line":{"color":"rgba(42, 63, 95, 1.0)"}},"colorway":["rgba(99, 110, 250, 1.0)","rgba(239, 85, 59, 1.0)","rgba(0, 204, 150, 1.0)","rgba(171, 99, 250, 1.0)","rgba(255, 161, 90, 1.0)","rgba(25, 211, 243, 1.0)","rgba(255, 102, 146, 1.0)","rgba(182, 232, 128, 1.0)","rgba(255, 151, 255, 1.0)","rgba(254, 203, 82, 1.0)"]},"data":{"bar":[{"marker":{"line":{"color":"rgba(229, 236, 246, 1.0)","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"error_x":{"color":"rgba(42, 63, 95, 1.0)"},"error_y":{"color":"rgba(42, 63, 95, 1.0)"}}],"barpolar":[{"marker":{"line":{"color":"rgba(229, 236, 246, 1.0)","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}}}],"carpet":[{"aaxis":{"linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","endlinecolor":"rgba(42, 63, 95, 1.0)","minorgridcolor":"rgba(255, 255, 255, 1.0)","startlinecolor":"rgba(42, 63, 95, 1.0)"},"baxis":{"linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","endlinecolor":"rgba(42, 63, 95, 1.0)","minorgridcolor":"rgba(255, 255, 255, 1.0)","startlinecolor":"rgba(42, 63, 95, 1.0)"}}],"choropleth":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contour":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"colorbar":{"outlinewidth":0.0,"ticks":""}}],"heatmap":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}}}],"histogram2d":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"histogram2dcontour":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"colorbar":{"outlinewidth":0.0,"ticks":""}}],"parcoords":[{"line":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"pie":[{"automargin":true}],"scatter":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scatter3d":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}},"line":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scattercarpet":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scattergeo":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scattergl":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scattermapbox":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scatterpolar":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scatterpolargl":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scatterternary":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"surface":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"table":[{"cells":{"fill":{"color":"rgba(235, 240, 248, 1.0)"},"line":{"color":"rgba(255, 255, 255, 1.0)"}},"header":{"fill":{"color":"rgba(200, 212, 227, 1.0)"},"line":{"color":"rgba(255, 255, 255, 1.0)"}}}]}},"title":{"text":"lynx"},"xaxis":{},"yaxis":{},"xaxis2":{},"yaxis2":{},"grid":{"rows":2,"columns":1,"pattern":"independent"}};
    var config = {"responsive":true};
    Plotly.newPlot('b0ac8c02-5183-4806-8e61-945e6561d243', data, layout, config);
};
renderPlotly_b0ac8c02518348068e61945e6561d243();
</script></div>

### Fitting to real data

First, we must load in the real data, which is in a CSV file. Here, we will use
the FSharp.Data type provider to read in the CSV file (see [the FSharp.Data docs](http://fsprojects.github.io/FSharp.Data/library/CsvProvider.html)
for further information on how to use the library). We place the raw data into
a Bristlecone `TimeSeries` type using `TimeSeries.fromObservations`:



In [9]:
[<Literal>]
let ResolutionFolder = __SOURCE_DIRECTORY__

type PopulationData = FSharp.Data.CsvProvider<"data/lynx-hare.csv", ResolutionFolder=ResolutionFolder>

let data =
    let csv = PopulationData.Load(__SOURCE_DIRECTORY__ + "/data/lynx-hare.csv")

    [ (code "hare").Value, Time.TimeSeries.fromObservations (csv.Rows |> Seq.map (fun r -> float r.Hare, r.Year))
      (code "lynx").Value, Time.TimeSeries.fromObservations (csv.Rows |> Seq.map (fun r -> float r.Lynx, r.Year)) ]
    |> Map.ofList


map  [(ShortCode "hare",    FixedTimeSeries  ((19.58, 1/1/1845 12:00:00 AM),   TimeSteps     [|(19.6, 365.00:00:00); (19.61, 365.00:00:00); (11.99, 365.00:00:00);       (28.04, 366.00:00:00); (58.0, 365.00:00:00); (74.6, 365.00:00:00);       (75.09, 365.00:00:00); (88.48, 366.00:00:00); (61.28, 365.00:00:00);       (74.67, 365.00:00:00); (88.06, 365.00:00:00); (68.51, 366.00:00:00);       (32.19, 365.00:00:00); (12.64, 365.00:00:00); (21.49, 365.00:00:00);       (30.35, 366.00:00:00); (2.18, 365.00:00:00); (152.65, 365.00:00:00);       (148.36, 365.00:00:00); (85.81, 366.00:00:00); (41.41, 365.00:00:00);       (14.75, 365.00:00:00); (2.28, 365.00:00:00); (5.91, 366.00:00:00);       (9.95, 365.00:00:00); (10.44, 365.00:00:00); (70.64, 365.00:00:00);       (50.12, 366.00:00:00); (50.13, 365.00:00:00); (101.25, 365.00:00:00);       (97.12, 365.00:00:00); (86.51, 366.00:00:00); (72.17, 365.00:00:00);       (38.32, 365.00:00:00); (10.11, 365.00:00:00); (7.74, 366.00:00:00);       (9.67, 365

Once the data are in Bristlecone `TimeSeries` we can run `Bristlecone.fit`, which is
the main fitting function of the Bristlecone library.



In [11]:
let endCondition = Optimisation.EndConditions.afterIteration 10000

let result = ``predator-prey`` |> Bristlecone.tryFit engine endCondition data


Ok  { ResultId = 07ee8ae5-dcac-4d0e-ba17-4f66386cbc82    Likelihood = 10254.13307    Parameters =     Pool       (map          [(ShortCode "α",            Parameter (Unconstrained, Detached, Estimated 1.111650885));           (ShortCode "β",            Parameter (Unconstrained, Detached, Estimated 0.04622625795));           (ShortCode "γ",            Parameter (Unconstrained, Detached, Estimated 0.00695142448));           (ShortCode "δ",            Parameter (Unconstrained, Detached, Estimated 0.2742201594));           (ShortCode "ρ",            Parameter (Unconstrained, Detached, Estimated 0.4773785127));           (ShortCode "σ[x]",            Parameter (PositiveOnly, Detached, Estimated 4.157612651));           (ShortCode "σ[y]",            Parameter (PositiveOnly, Detached, Estimated 2.489175895))])    Series =     map       [(ShortCode "hare",         FixedTimeSeries  (({ Fit = 16.33342445      Obs = 19.58 }, 1/1/1845 12:00:00 AM),   TimeSteps     [|({ Fit = 16.46830027          O

### Inspecting the model fit

The `Bristlecone.fit` function returns an `EstimationResult`, which contains some
key information that may be used to inspect the model fit:

* Likelihood. The minimum likelihood identified during optimisation.

* Parameters. The parameter set (**θ**) identified at the minimum likelihood.

* Series. A TimeSeries for each variable in the model, which at each time point contains paired Modelled-Observed values.

* Trace. The likelihood and **θ** that occurred at each step in optimisation, with the latest first.

* Internal Dynamics. Not relevant for this simple model.

First, we can use the `Series` to inspect by eye the model fit versus the observed time-series:

<div><div id="61a9957b-68e2-41a7-8f28-fae759c3522f"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_61a9957b68e241a78f28fae759c3522f = function() {
    var data = [{"type":"scatter","name":"Modelled","mode":"lines","x":["1845-01-01T00:00:00+00:00","1846-01-01T00:00:00+00:00","1847-01-01T00:00:00+00:00","1848-01-01T00:00:00+00:00","1849-01-01T00:00:00+00:00","1850-01-01T00:00:00+00:00","1851-01-01T00:00:00+00:00","1852-01-01T00:00:00+00:00","1853-01-01T00:00:00+00:00","1854-01-01T00:00:00+00:00","1855-01-01T00:00:00+00:00","1856-01-01T00:00:00+00:00","1857-01-01T00:00:00+00:00","1858-01-01T00:00:00+00:00","1859-01-01T00:00:00+00:00","1860-01-01T00:00:00+00:00","1861-01-01T00:00:00+00:00","1862-01-01T00:00:00+00:00","1863-01-01T00:00:00+00:00","1864-01-01T00:00:00+00:00","1865-01-01T00:00:00+00:00","1866-01-01T00:00:00+00:00","1867-01-01T00:00:00+00:00","1868-01-01T00:00:00+00:00","1869-01-01T00:00:00+00:00","1870-01-01T00:00:00+00:00","1871-01-01T00:00:00+00:00","1872-01-01T00:00:00+00:00","1873-01-01T00:00:00+00:00","1874-01-01T00:00:00+00:00","1875-01-01T00:00:00+00:00","1876-01-01T00:00:00+00:00","1877-01-01T00:00:00+00:00","1878-01-01T00:00:00+00:00","1879-01-01T00:00:00+00:00","1880-01-01T00:00:00+00:00","1881-01-01T00:00:00+00:00","1882-01-01T00:00:00+00:00","1883-01-01T00:00:00+00:00","1884-01-01T00:00:00+00:00","1885-01-01T00:00:00+00:00","1886-01-01T00:00:00+00:00","1887-01-01T00:00:00+00:00","1888-01-01T00:00:00+00:00","1889-01-01T00:00:00+00:00","1890-01-01T00:00:00+00:00","1891-01-01T00:00:00+00:00","1892-01-01T00:00:00+00:00","1893-01-01T00:00:00+00:00","1894-01-01T00:00:00+00:00","1895-01-01T00:00:00+00:00","1896-01-01T00:00:00+00:00","1897-01-01T00:00:00+00:00","1898-01-01T00:00:00+00:00","1899-01-01T00:00:00+00:00","1900-01-01T00:00:00+00:00","1901-01-01T00:00:00+00:00","1902-01-01T00:00:00+00:00","1903-01-01T00:00:00+00:00","1904-01-01T00:00:00+00:00","1905-01-01T00:00:00+00:00","1906-01-01T00:00:00+00:00","1907-01-01T00:00:00+00:00","1908-01-01T00:00:00+00:00","1909-01-01T00:00:00+00:00","1910-01-01T00:00:00+00:00","1911-01-01T00:00:00+00:00","1912-01-01T00:00:00+00:00","1913-01-01T00:00:00+00:00","1914-01-01T00:00:00+00:00","1915-01-01T00:00:00+00:00","1916-01-01T00:00:00+00:00","1917-01-01T00:00:00+00:00","1918-01-01T00:00:00+00:00","1919-01-01T00:00:00+00:00","1920-01-01T00:00:00+00:00","1921-01-01T00:00:00+00:00","1922-01-01T00:00:00+00:00","1923-01-01T00:00:00+00:00","1924-01-01T00:00:00+00:00","1925-01-01T00:00:00+00:00","1926-01-01T00:00:00+00:00","1927-01-01T00:00:00+00:00","1928-01-01T00:00:00+00:00","1929-01-01T00:00:00+00:00","1930-01-01T00:00:00+00:00","1931-01-01T00:00:00+00:00","1932-01-01T00:00:00+00:00","1933-01-01T00:00:00+00:00","1934-01-01T00:00:00+00:00","1935-01-01T00:00:00+00:00"],"y":[16.3334244504073,16.468300268540588,19.513411740239672,26.082965880739795,37.4335708679247,54.15715864764695,72.1910784000335,78.34268973048137,62.9761379429233,40.39195988099477,25.457470408318194,18.41507238292718,16.1260966418637,16.977523477381542,20.806697833925387,28.434653662852597,41.13780507216576,58.849493523759406,75.41778303076036,76.27910998944635,57.12620644132366,35.79555390983851,23.13401934340716,17.52723188248919,16.104815954361605,17.670152238946702,22.341042378337903,31.117942943980527,45.193792399909036,63.532219020946165,77.6150157743984,72.80035251566464,51.2162286018598,31.795075696518907,21.23830365524455,16.886111333345674,16.26423149590901,18.557816649084096,24.140464982496916,34.15509094099568,49.56644191094226,68.00021989250325,78.51425044032848,68.14246304174183,45.54597065796235,28.384455504299073,19.718378027231278,16.46832183517209,16.603838982413315,19.65597401084731,26.231076244039066,37.56171216320651,54.187488050382214,71.9953426344211,77.92546894664551,62.65376528775162,40.325806604577345,25.52673099455734,18.527697874283728,16.256930671600376,17.12771239243432,20.983844331162683,28.640112016444704,41.342179270357946,58.94601595567869,75.21933335158859,75.78735530126244,56.73277541054889,35.67580463109414,23.16928145279557,17.62654104242187,16.240784590248907,17.844107863728144,22.56393720488873,31.393913674843656,45.482906524994064,63.679311278239936,77.36231615589367,72.1990420029374,50.760746600392146,31.64145965777175,21.25498910367011,16.982691285189283,16.41415131848251,18.765312395550904,24.421464356043835,34.51494432432625,49.9436244163832,68.16689414465972,78.14759989455658,67.41722970288711],"marker":{},"line":{},"xaxis":"x","yaxis":"y"},{"type":"scatter","name":"Observed","mode":"lines","x":["1845-01-01T00:00:00+00:00","1846-01-01T00:00:00+00:00","1847-01-01T00:00:00+00:00","1848-01-01T00:00:00+00:00","1849-01-01T00:00:00+00:00","1850-01-01T00:00:00+00:00","1851-01-01T00:00:00+00:00","1852-01-01T00:00:00+00:00","1853-01-01T00:00:00+00:00","1854-01-01T00:00:00+00:00","1855-01-01T00:00:00+00:00","1856-01-01T00:00:00+00:00","1857-01-01T00:00:00+00:00","1858-01-01T00:00:00+00:00","1859-01-01T00:00:00+00:00","1860-01-01T00:00:00+00:00","1861-01-01T00:00:00+00:00","1862-01-01T00:00:00+00:00","1863-01-01T00:00:00+00:00","1864-01-01T00:00:00+00:00","1865-01-01T00:00:00+00:00","1866-01-01T00:00:00+00:00","1867-01-01T00:00:00+00:00","1868-01-01T00:00:00+00:00","1869-01-01T00:00:00+00:00","1870-01-01T00:00:00+00:00","1871-01-01T00:00:00+00:00","1872-01-01T00:00:00+00:00","1873-01-01T00:00:00+00:00","1874-01-01T00:00:00+00:00","1875-01-01T00:00:00+00:00","1876-01-01T00:00:00+00:00","1877-01-01T00:00:00+00:00","1878-01-01T00:00:00+00:00","1879-01-01T00:00:00+00:00","1880-01-01T00:00:00+00:00","1881-01-01T00:00:00+00:00","1882-01-01T00:00:00+00:00","1883-01-01T00:00:00+00:00","1884-01-01T00:00:00+00:00","1885-01-01T00:00:00+00:00","1886-01-01T00:00:00+00:00","1887-01-01T00:00:00+00:00","1888-01-01T00:00:00+00:00","1889-01-01T00:00:00+00:00","1890-01-01T00:00:00+00:00","1891-01-01T00:00:00+00:00","1892-01-01T00:00:00+00:00","1893-01-01T00:00:00+00:00","1894-01-01T00:00:00+00:00","1895-01-01T00:00:00+00:00","1896-01-01T00:00:00+00:00","1897-01-01T00:00:00+00:00","1898-01-01T00:00:00+00:00","1899-01-01T00:00:00+00:00","1900-01-01T00:00:00+00:00","1901-01-01T00:00:00+00:00","1902-01-01T00:00:00+00:00","1903-01-01T00:00:00+00:00","1904-01-01T00:00:00+00:00","1905-01-01T00:00:00+00:00","1906-01-01T00:00:00+00:00","1907-01-01T00:00:00+00:00","1908-01-01T00:00:00+00:00","1909-01-01T00:00:00+00:00","1910-01-01T00:00:00+00:00","1911-01-01T00:00:00+00:00","1912-01-01T00:00:00+00:00","1913-01-01T00:00:00+00:00","1914-01-01T00:00:00+00:00","1915-01-01T00:00:00+00:00","1916-01-01T00:00:00+00:00","1917-01-01T00:00:00+00:00","1918-01-01T00:00:00+00:00","1919-01-01T00:00:00+00:00","1920-01-01T00:00:00+00:00","1921-01-01T00:00:00+00:00","1922-01-01T00:00:00+00:00","1923-01-01T00:00:00+00:00","1924-01-01T00:00:00+00:00","1925-01-01T00:00:00+00:00","1926-01-01T00:00:00+00:00","1927-01-01T00:00:00+00:00","1928-01-01T00:00:00+00:00","1929-01-01T00:00:00+00:00","1930-01-01T00:00:00+00:00","1931-01-01T00:00:00+00:00","1932-01-01T00:00:00+00:00","1933-01-01T00:00:00+00:00","1934-01-01T00:00:00+00:00","1935-01-01T00:00:00+00:00"],"y":[19.58,19.6,19.61,11.99,28.04,58.0,74.6,75.09,88.48,61.28,74.67,88.06,68.51,32.19,12.64,21.49,30.35,2.18,152.65,148.36,85.81,41.41,14.75,2.28,5.91,9.95,10.44,70.64,50.12,50.13,101.25,97.12,86.51,72.17,38.32,10.11,7.74,9.67,43.12,52.21,134.85,134.86,103.79,46.1,15.03,24.2,41.65,52.34,53.78,70.4,85.81,56.69,16.59,6.16,2.3,12.82,4.72,4.73,37.22,69.72,57.78,28.68,23.37,21.54,26.34,53.1,68.48,75.58,57.92,40.97,24.95,12.59,4.97,4.5,11.21,56.6,69.63,77.74,80.53,73.38,36.93,4.64,2.54,1.8,2.39,4.23,19.52,82.11,89.76,81.66,15.76],"marker":{},"line":{},"xaxis":"x","yaxis":"y"},{"type":"scatter","name":"Modelled","mode":"lines","x":["1845-01-01T00:00:00+00:00","1846-01-01T00:00:00+00:00","1847-01-01T00:00:00+00:00","1848-01-01T00:00:00+00:00","1849-01-01T00:00:00+00:00","1850-01-01T00:00:00+00:00","1851-01-01T00:00:00+00:00","1852-01-01T00:00:00+00:00","1853-01-01T00:00:00+00:00","1854-01-01T00:00:00+00:00","1855-01-01T00:00:00+00:00","1856-01-01T00:00:00+00:00","1857-01-01T00:00:00+00:00","1858-01-01T00:00:00+00:00","1859-01-01T00:00:00+00:00","1860-01-01T00:00:00+00:00","1861-01-01T00:00:00+00:00","1862-01-01T00:00:00+00:00","1863-01-01T00:00:00+00:00","1864-01-01T00:00:00+00:00","1865-01-01T00:00:00+00:00","1866-01-01T00:00:00+00:00","1867-01-01T00:00:00+00:00","1868-01-01T00:00:00+00:00","1869-01-01T00:00:00+00:00","1870-01-01T00:00:00+00:00","1871-01-01T00:00:00+00:00","1872-01-01T00:00:00+00:00","1873-01-01T00:00:00+00:00","1874-01-01T00:00:00+00:00","1875-01-01T00:00:00+00:00","1876-01-01T00:00:00+00:00","1877-01-01T00:00:00+00:00","1878-01-01T00:00:00+00:00","1879-01-01T00:00:00+00:00","1880-01-01T00:00:00+00:00","1881-01-01T00:00:00+00:00","1882-01-01T00:00:00+00:00","1883-01-01T00:00:00+00:00","1884-01-01T00:00:00+00:00","1885-01-01T00:00:00+00:00","1886-01-01T00:00:00+00:00","1887-01-01T00:00:00+00:00","1888-01-01T00:00:00+00:00","1889-01-01T00:00:00+00:00","1890-01-01T00:00:00+00:00","1891-01-01T00:00:00+00:00","1892-01-01T00:00:00+00:00","1893-01-01T00:00:00+00:00","1894-01-01T00:00:00+00:00","1895-01-01T00:00:00+00:00","1896-01-01T00:00:00+00:00","1897-01-01T00:00:00+00:00","1898-01-01T00:00:00+00:00","1899-01-01T00:00:00+00:00","1900-01-01T00:00:00+00:00","1901-01-01T00:00:00+00:00","1902-01-01T00:00:00+00:00","1903-01-01T00:00:00+00:00","1904-01-01T00:00:00+00:00","1905-01-01T00:00:00+00:00","1906-01-01T00:00:00+00:00","1907-01-01T00:00:00+00:00","1908-01-01T00:00:00+00:00","1909-01-01T00:00:00+00:00","1910-01-01T00:00:00+00:00","1911-01-01T00:00:00+00:00","1912-01-01T00:00:00+00:00","1913-01-01T00:00:00+00:00","1914-01-01T00:00:00+00:00","1915-01-01T00:00:00+00:00","1916-01-01T00:00:00+00:00","1917-01-01T00:00:00+00:00","1918-01-01T00:00:00+00:00","1919-01-01T00:00:00+00:00","1920-01-01T00:00:00+00:00","1921-01-01T00:00:00+00:00","1922-01-01T00:00:00+00:00","1923-01-01T00:00:00+00:00","1924-01-01T00:00:00+00:00","1925-01-01T00:00:00+00:00","1926-01-01T00:00:00+00:00","1927-01-01T00:00:00+00:00","1928-01-01T00:00:00+00:00","1929-01-01T00:00:00+00:00","1930-01-01T00:00:00+00:00","1931-01-01T00:00:00+00:00","1932-01-01T00:00:00+00:00","1933-01-01T00:00:00+00:00","1934-01-01T00:00:00+00:00","1935-01-01T00:00:00+00:00"],"y":[25.853281075702043,21.989128063442376,18.909113502734954,16.803476525109755,15.880741102016552,16.55540313400398,19.56979277236063,25.38730188728279,31.852012356118784,34.62446033120575,32.914034282402945,29.038132891574854,24.83719774613897,21.147944053073545,18.2981619757239,16.46480200089931,15.890548149095386,17.067684899026332,20.783057778910084,27.10394657404192,32.99130858529479,34.51899569257134,32.04316670771742,27.96911008275631,23.852004300949442,20.35652947245353,17.747925860050614,16.202490998773953,16.00856687970375,17.742736649234917,22.18041185522542,28.803591899742667,33.828917106080134,34.156739482831206,31.07801058053426,26.899328825488993,22.904484615438236,19.618401277621707,17.262186166719143,16.023451170699445,16.24783096605175,18.596091627680803,23.740388271068273,30.399833578812913,34.34104455884833,33.57853300939439,30.048552538631736,25.842239988866297,22.000319875773616,18.93699126700271,16.845359413374332,15.936046764321222,16.62326672685319,19.6399488192892,25.42143034690841,31.80693703284657,34.52942445219327,32.82661244725249,28.980194080191573,24.80873108748972,21.14427514048857,18.31574652328427,16.502641124756817,15.950319857501507,17.151367716941603,20.879480225851246,27.160682047971424,32.95165113163682,34.416727235232784,31.941274595050245,27.894080878500034,23.807603437000566,20.340442838927313,17.75827349506382,16.240166222070624,16.078125458859887,17.849216085571918,22.30782513573409,28.87712540036336,33.78299230107634,34.03948469937905,30.958957626694236,26.807651930119707,22.846086478189115,19.59257155150687,17.268607327997536,16.065304366572875,16.333267303916532,18.732714905364748,23.900624226916584,30.479663225160046],"marker":{},"line":{},"xaxis":"x2","yaxis":"y2"},{"type":"scatter","name":"Observed","mode":"lines","x":["1845-01-01T00:00:00+00:00","1846-01-01T00:00:00+00:00","1847-01-01T00:00:00+00:00","1848-01-01T00:00:00+00:00","1849-01-01T00:00:00+00:00","1850-01-01T00:00:00+00:00","1851-01-01T00:00:00+00:00","1852-01-01T00:00:00+00:00","1853-01-01T00:00:00+00:00","1854-01-01T00:00:00+00:00","1855-01-01T00:00:00+00:00","1856-01-01T00:00:00+00:00","1857-01-01T00:00:00+00:00","1858-01-01T00:00:00+00:00","1859-01-01T00:00:00+00:00","1860-01-01T00:00:00+00:00","1861-01-01T00:00:00+00:00","1862-01-01T00:00:00+00:00","1863-01-01T00:00:00+00:00","1864-01-01T00:00:00+00:00","1865-01-01T00:00:00+00:00","1866-01-01T00:00:00+00:00","1867-01-01T00:00:00+00:00","1868-01-01T00:00:00+00:00","1869-01-01T00:00:00+00:00","1870-01-01T00:00:00+00:00","1871-01-01T00:00:00+00:00","1872-01-01T00:00:00+00:00","1873-01-01T00:00:00+00:00","1874-01-01T00:00:00+00:00","1875-01-01T00:00:00+00:00","1876-01-01T00:00:00+00:00","1877-01-01T00:00:00+00:00","1878-01-01T00:00:00+00:00","1879-01-01T00:00:00+00:00","1880-01-01T00:00:00+00:00","1881-01-01T00:00:00+00:00","1882-01-01T00:00:00+00:00","1883-01-01T00:00:00+00:00","1884-01-01T00:00:00+00:00","1885-01-01T00:00:00+00:00","1886-01-01T00:00:00+00:00","1887-01-01T00:00:00+00:00","1888-01-01T00:00:00+00:00","1889-01-01T00:00:00+00:00","1890-01-01T00:00:00+00:00","1891-01-01T00:00:00+00:00","1892-01-01T00:00:00+00:00","1893-01-01T00:00:00+00:00","1894-01-01T00:00:00+00:00","1895-01-01T00:00:00+00:00","1896-01-01T00:00:00+00:00","1897-01-01T00:00:00+00:00","1898-01-01T00:00:00+00:00","1899-01-01T00:00:00+00:00","1900-01-01T00:00:00+00:00","1901-01-01T00:00:00+00:00","1902-01-01T00:00:00+00:00","1903-01-01T00:00:00+00:00","1904-01-01T00:00:00+00:00","1905-01-01T00:00:00+00:00","1906-01-01T00:00:00+00:00","1907-01-01T00:00:00+00:00","1908-01-01T00:00:00+00:00","1909-01-01T00:00:00+00:00","1910-01-01T00:00:00+00:00","1911-01-01T00:00:00+00:00","1912-01-01T00:00:00+00:00","1913-01-01T00:00:00+00:00","1914-01-01T00:00:00+00:00","1915-01-01T00:00:00+00:00","1916-01-01T00:00:00+00:00","1917-01-01T00:00:00+00:00","1918-01-01T00:00:00+00:00","1919-01-01T00:00:00+00:00","1920-01-01T00:00:00+00:00","1921-01-01T00:00:00+00:00","1922-01-01T00:00:00+00:00","1923-01-01T00:00:00+00:00","1924-01-01T00:00:00+00:00","1925-01-01T00:00:00+00:00","1926-01-01T00:00:00+00:00","1927-01-01T00:00:00+00:00","1928-01-01T00:00:00+00:00","1929-01-01T00:00:00+00:00","1930-01-01T00:00:00+00:00","1931-01-01T00:00:00+00:00","1932-01-01T00:00:00+00:00","1933-01-01T00:00:00+00:00","1934-01-01T00:00:00+00:00","1935-01-01T00:00:00+00:00"],"y":[30.09,45.15,49.15,39.52,21.23,8.42,5.56,5.08,10.17,19.6,32.91,34.38,29.59,21.3,13.69,7.65,4.08,4.09,14.33,38.22,60.78,70.77,72.77,42.68,16.39,9.83,5.8,5.26,18.91,30.95,31.18,46.34,45.77,44.15,36.33,12.03,12.6,18.34,35.14,43.77,65.69,79.35,51.65,32.59,22.45,16.16,14.12,20.38,33.33,46.0,51.41,46.43,33.68,18.01,8.86,7.13,9.47,14.86,31.47,60.57,63.51,54.7,6.3,3.41,5.44,11.65,20.35,32.88,39.55,43.36,40.83,30.36,17.18,6.82,3.19,3.52,9.94,20.3,31.99,42.36,49.08,53.99,52.25,37.7,19.14,6.98,8.31,16.01,24.82,29.7,35.4],"marker":{},"line":{},"xaxis":"x2","yaxis":"y2"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"title":{"x":0.05},"font":{"color":"rgba(42, 63, 95, 1.0)"},"paper_bgcolor":"rgba(255, 255, 255, 1.0)","plot_bgcolor":"rgba(229, 236, 246, 1.0)","autotypenumbers":"strict","colorscale":{"diverging":[[0.0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1.0,"#276419"]],"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]},"hovermode":"closest","hoverlabel":{"align":"left"},"coloraxis":{"colorbar":{"outlinewidth":0.0,"ticks":""}},"geo":{"showland":true,"landcolor":"rgba(229, 236, 246, 1.0)","showlakes":true,"lakecolor":"rgba(255, 255, 255, 1.0)","subunitcolor":"rgba(255, 255, 255, 1.0)","bgcolor":"rgba(255, 255, 255, 1.0)"},"mapbox":{"style":"light"},"polar":{"bgcolor":"rgba(229, 236, 246, 1.0)","radialaxis":{"linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","ticks":""},"angularaxis":{"linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","ticks":""}},"scene":{"xaxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","gridwidth":2.0,"zerolinecolor":"rgba(255, 255, 255, 1.0)","backgroundcolor":"rgba(229, 236, 246, 1.0)","showbackground":true},"yaxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","gridwidth":2.0,"zerolinecolor":"rgba(255, 255, 255, 1.0)","backgroundcolor":"rgba(229, 236, 246, 1.0)","showbackground":true},"zaxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","gridwidth":2.0,"zerolinecolor":"rgba(255, 255, 255, 1.0)","backgroundcolor":"rgba(229, 236, 246, 1.0)","showbackground":true}},"ternary":{"aaxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)"},"baxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)"},"caxis":{"ticks":"","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)"},"bgcolor":"rgba(229, 236, 246, 1.0)"},"xaxis":{"title":{"standoff":15},"ticks":"","automargin":"height+width+left+right+top+bottom","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","zerolinecolor":"rgba(255, 255, 255, 1.0)","zerolinewidth":2.0},"yaxis":{"title":{"standoff":15},"ticks":"","automargin":"height+width+left+right+top+bottom","linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","zerolinecolor":"rgba(255, 255, 255, 1.0)","zerolinewidth":2.0},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"shapedefaults":{"line":{"color":"rgba(42, 63, 95, 1.0)"}},"colorway":["rgba(99, 110, 250, 1.0)","rgba(239, 85, 59, 1.0)","rgba(0, 204, 150, 1.0)","rgba(171, 99, 250, 1.0)","rgba(255, 161, 90, 1.0)","rgba(25, 211, 243, 1.0)","rgba(255, 102, 146, 1.0)","rgba(182, 232, 128, 1.0)","rgba(255, 151, 255, 1.0)","rgba(254, 203, 82, 1.0)"]},"data":{"bar":[{"marker":{"line":{"color":"rgba(229, 236, 246, 1.0)","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"error_x":{"color":"rgba(42, 63, 95, 1.0)"},"error_y":{"color":"rgba(42, 63, 95, 1.0)"}}],"barpolar":[{"marker":{"line":{"color":"rgba(229, 236, 246, 1.0)","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}}}],"carpet":[{"aaxis":{"linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","endlinecolor":"rgba(42, 63, 95, 1.0)","minorgridcolor":"rgba(255, 255, 255, 1.0)","startlinecolor":"rgba(42, 63, 95, 1.0)"},"baxis":{"linecolor":"rgba(255, 255, 255, 1.0)","gridcolor":"rgba(255, 255, 255, 1.0)","endlinecolor":"rgba(42, 63, 95, 1.0)","minorgridcolor":"rgba(255, 255, 255, 1.0)","startlinecolor":"rgba(42, 63, 95, 1.0)"}}],"choropleth":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contour":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"colorbar":{"outlinewidth":0.0,"ticks":""}}],"heatmap":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}}}],"histogram2d":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"histogram2dcontour":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"colorbar":{"outlinewidth":0.0,"ticks":""}}],"parcoords":[{"line":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"pie":[{"automargin":true}],"scatter":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scatter3d":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}},"line":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scattercarpet":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scattergeo":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scattergl":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scattermapbox":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scatterpolar":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scatterpolargl":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"scatterternary":[{"marker":{"colorbar":{"outlinewidth":0.0,"ticks":""}}}],"surface":[{"colorbar":{"outlinewidth":0.0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"table":[{"cells":{"fill":{"color":"rgba(235, 240, 248, 1.0)"},"line":{"color":"rgba(255, 255, 255, 1.0)"}},"header":{"fill":{"color":"rgba(200, 212, 227, 1.0)"},"line":{"color":"rgba(255, 255, 255, 1.0)"}}}]}},"title":{"text":"lynx"},"xaxis":{},"yaxis":{},"xaxis2":{},"yaxis2":{},"grid":{"rows":2,"columns":1,"pattern":"independent"}};
    var config = {"responsive":true};
    Plotly.newPlot('61a9957b-68e2-41a7-8f28-fae759c3522f', data, layout, config);
};
renderPlotly_61a9957b68e241a78f28fae759c3522f();
</script></div>

**NB: this documentation is auto-generated so we cannot comment directly on the randomly generated scenario.**

Next, we can examine the traces to see how parameter values evolved over the course of
the optimisation routine:



In [14]:
Graphing.parameterTrace result


<!-- Plotly chart will be drawn inside this DIV -->